RLHF is a costly way of aligning LLM from model drifting. ORPO eliminates the reference model in the fine-tuning stage itself.

- transformers: A popular library for natural language processing (NLP) tasks, providing pre-trained models like LLAMA3.
- datasets: A library for loading and processing datasets, used for training and evaluation.
- accelerate: A library for accelerating training and inference of ML models, particularly useful for large models like LLAMA3.
- peft: A library for parameter-efficient fine-tuning (PEFT) of pre-trained models, which allows for efficient adaptation to specific tasks.
- trl: A library for training and evaluating large language models, including LLAMA3.
- bitsandbytes: A library for efficient integer-precision optimization, used for faster and more memory-efficient training.
- wandb: A library for tracking and visualizing training runs, hyperparameters, and results, using Weights & Biases (W&B) as the backend.

In [1]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━

In [3]:
# Import the garbage collector interface
import gc

# Import operating system interfaces
import os

# Import the PyTorch library for working with deep learning models
import torch

# Import Weights & Biases to log and visualize the model training process
import wandb

# Import function to load datasets from the Hugging Face 'datasets' library
from datasets import load_dataset

# Import Google Colab user data utilities (not typically used outside Colab environments)
from google.colab import userdata

# Import LoraConfig, PeftModel, and prepare model for k-bit training utilities for model optimization
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# Import the AutoModelForCausalLM and AutoTokenizer for loading and using pre-trained models
# Import BitsAndBytesConfig for configuring training with lower precision to save memory
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline

# Import ORPOTrainer and ORPOConfig for training using ORPO optimization
# setup_chat_format utility function to setup prompt formats for chat-like tasks
from trl import ORPOTrainer, ORPOConfig, setup_chat_format



In [4]:
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
# Check if the CUDA-capable device's compute capability is at least 8.0
# This typically indicates support for more advanced features and higher precision types like bfloat16.
if torch.cuda.get_device_capability()[0] >= 8:
    print("using bitsandbytes")
    # Install the flash-attn package quietly without showing output.
    # This package provides efficient attention mechanisms that are optimized for newer GPU architectures.
    !pip install -qqq flash-attn
    # Set the attention implementation to use the 'flash_attention_2', which is optimized for newer GPUs.
    attn_implementation = "flash_attention_2"
    # Use bfloat16 as the data type for PyTorch tensors, which allows for faster computation and reduced memory usage
    # on GPUs that support this feature (newer architectures).
    torch_dtype = torch.bfloat16
else:
    # For GPUs with lower compute capabilities (less than 8.0), use a standard attention mechanism.
    attn_implementation = "eager"
    # Use float16 as the data type, which is supported on older GPU architectures and still offers reduced memory
    # usage compared to float32.
    torch_dtype = torch.float16


using bitsandbytes


In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
# Define the identifier for the base pre-trained model from the Hugging Face Model Hub.
base_model = "meta-llama/Meta-Llama-3-8B"
# Define the identifier for the new model to be fine-tuned or modified.
new_model = "OrpoLlama3-8B-FT"

# Configuration for using the BitsAndBytes library to quantize the model weights to 4-bit precision.
# This can help reduce model size and potentially increase inference speed with minimal impact on accuracy.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading of the model in 4-bit precision
    bnb_4bit_quant_type="nf4",  # Specify the type of quantization, here 'nf4' could stand for a specific 4-bit quantization type
    bnb_4bit_compute_dtype=torch_dtype,  # Set the data type for computation using the previously set torch_dtype
    bnb_4bit_quant_alpha_zero=True,  # This could be a parameter specific to the quantization process used in BitsAndBytes
)

# Configuration for the Lora (Low-Rank Adaptation) technique which modifies only a small part of the model's weights,
# making the training process more efficient and specialized.
peft_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor for the low-rank updates
    lora_dropout=0.05,  # Dropout rate to prevent overfitting in the adapted layers
    bias="none",  # Whether to include bias in the low-rank layers
    task_type="CAUSAL_LM",  # Type of the task, here causal language modeling
    target_modules=["up_proj","down_proj","gate_proj","q_proj", "v_proj"],  # Specify model components to apply Lora
)

# Load a tokenizer that matches the pre-trained base model, which is used to convert text input into a format suitable for the model.
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load the pre-trained causal language model from Hugging Face, applying the quantization settings.
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,  # Apply the previously defined BitsAndBytes configuration
    device_map="auto",  # Automatically map model layers to available CUDA devices if possible
    attn_implementation=attn_implementation  # Use the attention implementation based on device capability
)

# Format the model and tokenizer for conversational usage if necessary.
model, tokenizer = setup_chat_format(model, tokenizer)

# Prepare the model for training using the k-bit training technique, which likely involves further quantization or model adjustments.
model = prepare_model_for_kbit_training(model)


Unused kwargs: ['bnb_4bit_quant_alpha_zero']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [13]:

dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

In [14]:
dataset

[Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 61135
 }),
 Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 2000
 })]

In [18]:
# Import the necessary library to work with datasets.
from datasets import load_dataset

# Load the dataset named 'ultrafeedback_binarized' from Hugging Face's dataset repository.
# The dataset is split into two parts: 'train_prefs' and 'test_prefs'.
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# Define the number of training samples to be used in this experiment.
train_samples = 5000

# The original number of training samples in the dataset as provided by the dataset documentation or metadata.
original_train_samples = 61135

# Calculate the number of test samples to be used. The ratio of test samples to original training samples (2000/61135)
# is used to scale the number of test samples proportional to the new number of training samples.
test_samples = int((2000 / original_train_samples) * train_samples)

# Shuffle the first part of the dataset (training data) and select a subset equal to 'train_samples'.
# The seed is set to 42 to ensure reproducibility of the shuffle.
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))

# Shuffle the second part of the dataset (testing data) and select a subset equal to 'test_samples'.
# The seed is set to 42 to ensure reproducibility of the shuffle.
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))

# Print the number of samples in the training subset to verify the correct number has been selected.
print(f"Number of training samples: {len(train_subset)}")

# Print the number of samples in the testing subset to verify the correct number has been selected.
print(f"Number of test samples: {len(test_subset)}")


Number of training samples: 5000
Number of test samples: 163


In [19]:
import multiprocessing

In [22]:
import multiprocessing
from datasets import load_dataset

# Define a function to format text data in each row
def process(row):
    # Apply a chat template to the 'chosen' field without tokenizing it
    row["chosen"] = tokenizer.apply_chat_template(row['chosen'], tokenize=False)
    # Apply a chat template to the 'rejected' field without tokenizing it
    row["rejected"] = tokenizer.apply_chat_template(row['rejected'], tokenize=False)
    # Return the modified row
    return row

# Load the datasets specifying splits for training and testing
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# Set the number of training samples
train_samples = 5000


Map (num_proc=16):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/163 [00:00<?, ? examples/s]

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 5000
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 163
})]


In [24]:
# ORPO configuration setup as per the recommendations from the ORPO paper, specifically advising a lower learning rate compared to traditional Sparse Fine-Tuning (SFT) or Differentiable Prompt Optimization (DPO) techniques.

orpo_args = ORPOConfig(
    learning_rate=8e-6,  # Set a lower learning rate as recommended for more stable optimization
    beta=0.1,  # Beta parameter for optimization, specific to ORPO configurations
    lr_scheduler_type="linear",  # Use a linear learning rate scheduler to gradually decrease the learning rate
    max_length=1024,  # Maximum length of the sequences to be processed
    max_prompt_length=512,  # Maximum length allowed for prompts
    per_device_train_batch_size=2,  # Training batch size per device
    per_device_eval_batch_size=2,  # Evaluation batch size per device
    gradient_accumulation_steps=4,  # Number of steps to accumulate gradients before performing a backward/update pass
    optim="paged_adamw_8bit",  # Specify the optimizer with 8-bit precision enhancements
    max_steps=1000,  # Maximum number of training steps to run
    evaluation_strategy="steps",  # Evaluation is performed after a set number of steps
    eval_steps=100,  # Perform evaluations every 100 steps
    logging_steps=1,  # Log metrics after every step to keep a detailed training log
    warmup_steps=10,  # Number of steps to perform learning rate warmup
    report_to="wandb",  # Enable reporting to Weights & Biases to track experiments
    output_dir="./results/"  # Directory to save training outputs
)


In [25]:
# Initialize the ORPOTrainer with the specified model, training configuration, datasets, tokenizer, and PEFT configuration.
trainer = ORPOTrainer(
    model=model,  # The model to be trained; should be pre-loaded and configured
    args=orpo_args,  # Configuration for the ORPO training process, including learning rates, batch sizes, etc.
    train_dataset=dataset[0],  # The dataset to use for training, which should be pre-processed and ready for use
    eval_dataset=dataset[1],  # The dataset to use for evaluation during training to monitor performance and overfitting
    tokenizer=tokenizer,  # The tokenizer associated with the model, used for processing text into a format the model can understand
    peft_config=peft_config  # Configuration for Parametric Efficient Fine-Tuning (PEFT) that introduces new parameters in a way that increases efficiency
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/163 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [26]:
trainer.train()
trainer.save_model(new_model)

wandb: Currently logged in as: anish-gillella (anish-gillella-official). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

### How to merge LoRA adapters

In [28]:
#First we need to flush the memory
del trainer, model
gc.collect()
torch.cuda.empty_cache()

#Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto")

model, tokenizer = setup_chat_format(model, tokenizer)


#merge
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

#Pushing it on huggingface
model.push_to_hub(new_model,use_temp_dir=False)
tokenizer.push_to_hub(new_model,use_temp_dir=False)

NameError: name 'trainer' is not defined